In [17]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


llm = ChatOpenAI()

llm.invoke("Kim jest Natalia Piórkowska?")


AIMessage(content='Natalia Piórkowska jest polską aktorką teatralną i filmową. Znana głównie z ról w popularnych polskich serialach telewizyjnych, takich jak "M jak miłość" czy "Barwy szczęścia". Grała także w wielu przedstawieniach teatralnych, zdobywając uznanie widzów i krytyków. Jest cenioną artystką w Polsce.')

In [18]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "Jesteś ekspertem od osób pracujących na Politechnice Wrocławskiej."),
    ("user", "{input}")
])

In [19]:
chain = prompt | llm 

chain.invoke({"input": "Kim jest Natalia Piórkowska?"})

AIMessage(content='Natalia Piórkowska jest pracownikiem naukowym na Politechnice Wrocławskiej. Jest specjalistką w dziedzinie informatyki, zajmuje się m.in. sztuczną inteligencją, przetwarzaniem danych oraz analizą danych. Może prowadzić badania naukowe, wykłady oraz projekty związane z jej obszarem ekspertyzy. Jeśli chcesz uzyskać więcej informacji na temat konkretnych osiągnięć Natalii Piórkowskiej, sugeruję skontaktowanie się bezpośrednio z nią lub z odpowiednią jednostką na Politechnice Wrocławskiej.')

In [20]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "Kim jest Natalia Piórkowska?"})

'Natalia Piórkowska jest pracownikiem naukowym na Politechnice Wrocławskiej. Jest specjalistką w dziedzinie inżynierii środowiska. Może prowadzić badania, nauczać studentów oraz uczestniczyć w projektach badawczych związanych z ochroną środowiska i zrównoważonym rozwojem. Jeśli chcesz uzyskać więcej informacji na temat konkretnych osiągnięć lub zainteresowań naukowych Natalii Piórkowskiej, warto skontaktować się z nią bezpośrednio lub sprawdzić jej profil na stronie internetowej uczelni.'

In [26]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://kis.pwr.edu.pl/struktura/zaklad-inzynierii-oprogramowania-i-nauki-o-danych/pracownia-inzynierii-oprogramowania-i-nauki-o-danych")

docs = loader.load()
print(docs)

[Document(page_content='\n\nPracownia Inżynierii Oprogramowania i Nauki o Danych / Katedra Informatyki Stosowanej \n\n\n\n\n \n \n    \n\n\n\n\n\n\n\nTWOJA PRZEGLĄDARKA JEST NIEAKTUALNA.\nWykryliśmy, że używasz nieaktualnej przeglądarki, przez co nasz serwis może dla Ciebie działać niepoprawnie. Zalecamy aktualizację lub przejście na inną przeglądarkę. \n\n\n\n\n\n\n\n\n\nA\nA+\nA++\n\n\nA\nA\nA\nA\n\n\n\nPolskipl\n•\nEnglishen\n\n\n\n                        Logowanie                    \n\n\n\n\n\n\n\n\n\nSzukaj\n\n\n\n\n\n\nSzukaj \n\n\n \n\n\n\n \n\n\n \n\n\n\n\nimage/svg+xml\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                    \xa0\n                \n\n\n\n\n\n\n\n\nKatedra Informatyki Stosowanej\n\n\n\nMenu \n Powrót\n\nKatedraAktualnościDydaktykaBadaniaStrukturaPracownicyKontakt \n\n\n\n\n\n\n\n\n\nStrona główna\n\nStruktura\n\nZakład Inżynierii Oprogramowania i Nauki o Danych\n\n\n\n\n\n\n\n\nPracownia Inżynierii Oprogramowania i Nauki o Danych\n\n\n\n\n\n\n\n\nKierownik: dr h

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [31]:
from langchain_community.vectorstores.faiss import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [37]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Odpowiedz na poniższe pytanie jedynie używając informacji zawartych w poniższym tekście.

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [38]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever(search_kwargs={"k": 1})
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [40]:
response = retrieval_chain.invoke({"input": "Kim jest Natalia Piórkowska?"})


response["answer"]

'Natalia Piórkowska jest dr inżynierem oraz asystentem dydaktycznym.'

In [41]:
response["context"]

[Document(page_content='dr inż. Natalia PiórkowskaAsystent dydaktycznyPok.: D-2, 302a;\xa0Tel: 71-3203969;\xa0e-mail,\xa0dblp,\xa0DONA', metadata={'source': 'https://kis.pwr.edu.pl/struktura/zaklad-inzynierii-oprogramowania-i-nauki-o-danych/pracownia-inzynierii-oprogramowania-i-nauki-o-danych', 'title': 'Pracownia Inżynierii Oprogramowania i Nauki o Danych / Katedra Informatyki Stosowanej', 'language': 'pl'})]